In [ ]:
# import
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K  
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping
%matplotlib inline

# Read data
data = pd.read_csv('./filename.csv')


labels = pd.factorize(data.pop('species'))
y = labels[0]
_ = data.pop('id')
x = data

# Split data
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(x, y)
# train_x.shape, test_x.shape

mean = train_x.mean(axis=0)
std = train_x.std(axis=0)

# Data standardization
train_x_norm = (train_x - mean)/std
test_x_norm = (test_x - mean)/std

train_x_norm = np.expand_dims(train_x_norm, -1)
# train_x_norm.shape

test_x_norm = np.expand_dims(test_x_norm, -1)
# test_x_norm.shape



In [ ]:
# Definition model
def model_design(x):
    print(f"The number of neurons is : {int(x[0])}")
    K.clear_session()  # Clear a previously existing calculation diagram
    # Define the number of model layers
    inputs = layers.Input(shape=train_x_norm.shape[1:])
    z = layers.Conv1D(64, 11, activation='relu', padding='same')(inputs)
    z = layers.Conv1D(64, 11, activation='relu', padding='same')(z)
    z = layers.MaxPooling1D(3)(z)
    z = layers.Dropout(0.5)(z)
    
    z = layers.Conv1D(128, 11, activation='relu', padding='same')(z)
    z = layers.Conv1D(128, 11, activation='relu', padding='same')(z)
    z = layers.MaxPooling1D(1)(z)
    z = layers.Dropout(0.5)(z)
    
    z = layers.Conv1D(256, 11, activation='relu', padding='same')(z)
    z = layers.Conv1D(256, 11, activation='relu', padding='same')(z)
    z = layers.MaxPooling1D(1)(z)
    z = layers.Dropout(0.5)(z)
    
    z = layers.Conv1D(512, 11, activation='relu', padding='same')(z)
    z = layers.Conv1D(512, 11, activation='relu', padding='same')(z)
    z = layers.Dropout(0.5)(z)
    z = layers.GlobalAveragePooling1D()(z)
    z = layers.Dropout(0.5)(z)
    
    z = layers.Dense(256, activation='relu')(z)
    z = layers.Dropout(0.5)(z)
    z = layers.Dense(int(x[0]), activation='relu')(z)
    z = layers.Dense(3, activation='softmax')(z)

    model = tf.keras.Model(inputs=inputs, outputs=z)

    model.compile(optimizer=keras.optimizers.RMSprop(),
              loss='sparse_categorical_crossentropy',
              metrics=['acc']
             )
    model.summary()
    return model  # return model

                      
# Define a model training function
def best_model(x):
    model = model_design(x)  # Invoke the model design function
    history = model.fit(train_x_norm, train_y, epochs =300 ,
                    validation_data = (test_x_norm, test_y),
#                     callbacks = [tf_callback,callback],
                    batch_size = 256, verbose = 1)  

    val_loss = model.evaluate(test_x_norm, test_y, verbose=0)
    print(f"train loss: {val_loss}")

    return val_loss[0]


# Import pso particle swarm module
from pyswarm import pso  
# Defined upper and lower limits
lowerb = [32]
upperb = [64]

# The pso particle swarm optimization algorithm is invoked 
xopt, fopt = pso(func=best_model, lb=lowerb, ub=upperb, maxiter=1, swarmsize=2)
print('Optimal number of neurons：', int(xopt[0]))

# After executing this code, pso will optimize the number of neurons in the fully connected layer, and after several rounds of optimization, the optimal number of neurons is obtained.

In [ ]:
# Invoke model
model = model_design(x = [38])  # Input the optimized number of neurons into the model
history = model.fit(train_x_norm, train_y, epochs =600 ,
                validation_data = (test_x_norm, test_y),
#                     callbacks = [tf_callback,callback],
                batch_size = 256, verbose = 1)  

In [ ]:
# draw
plt.plot(history.epoch, history.history.get('acc'), 'y', label='Training acc')
plt.plot(history.epoch, history.history.get('val_acc'), 'b', label='Test acc')
plt.legend()